# TODO - debug

In [ ]:
decoder = tf.keras.models.Sequential()
for l in ae.layers[7:]:
    decoder.add(l)

In [ ]:
cluster1 = imgs[sc.labels_ == 2]
hid1, enc1 = ae.predict(cluster1[[0]])
hid2, enc2 = ae.predict(cluster1[[1]])

In [ ]:
def interpolate(x, y, num=100):
    return [(1 - z) * x + z * y for z in np.linspace(0,1,100)]

def draw_frame(ax, img, title, cmap="bone", channel=0):
    im = ax.imshow(img[0,:,:,channel], cmap=cmap)
    ax.set_title(title)
    return im

In [ ]:
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
# No joke repeating those ^^ prevents some matplotlib / jupyter error

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))

pixel_interpolation = interpolate(enc1, enc2)
encoded_interpolation = (decoder.predict(h) for h in interpolate(hid1, hid2))
pframes = [draw_frame(ax2, i, "pixel space interpolation") for i in pixel_interpolation]
eframes = [draw_frame(ax1, i, "hidden space interpolation") for i in encoded_interpolation]
ims = list(zip(pframes, eframes))

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=100)

if False: #save
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('interpolation.mp4', writer=writer)


### Interpolation walk through a cluster

In [ ]:
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook
if True:
    examples = 10
    cimgs = imgs[sc.labels_ == 5]
    cimgs = cimgs[np.random.choice(cimgs.shape[0], examples)]

    fig, ax = plt.subplots(figsize=(10,10))
    ims = []

    hids = [ae.predict(cimgs[[i]])[0] for i in range(examples)]

    for i in range(examples):
        j = (i + 1) % examples
        interp = (decoder.predict(h) for h in interpolate(hids[i], hids[j], 10))
        ims.extend([[draw_frame(ax, i, "")] for i in interp])

    ani = animation.ArtistAnimation(fig, ims, interval=100)
    
#     
# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
# ani.save('rw_interpolation.mp4', writer=writer)

This is pretty expensive to compute and turns out to not give much insight to the clusters

In [ ]:
%matplotlib inline

examples = 50
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(16, 8))

for i in range(10):
    cimgs = imgs[km.labels_ == i]
    cimgs = cimgs[np.random.choice(cimgs.shape[0], examples)]
    
    m = ae.predict(cimgs)[0]
    m = m.mean(axis=0)
    m = np.expand_dims(m, axis=0)
    m = decoder.predict(m)
    
    a = ax[i // 5, i % 5]
    a.set_title("decoded cluster %d mean"%i)
    a.imshow(m[0,:,:,0], cmap="bone")
    a.set_xticks([]); a.set_yticks([])

It seems taking the mean of the cluster and decoding does not produce meaningful results. This suggests that the AE latent manifold is not convex